In [3]:
import matplotlib.pyplot as plt    #Importamos pyplot de librería matplotlib. Lo vamos a utilizar para graficar.
import seaborn as sns              #Importamos la librería Seaborn. La vamos a utilizar para graficar.
import numpy as np                 #Importamos la librería numpy para manipular arreglos.
import pandas as pd
import os
from pathlib import Path

# **Anime recommendation**

#### **Carga de datos**

In [101]:
path = Path(os.getcwd())
path = str(path.parent.absolute())
path = path+"/datos/anime.csv"
dfAnime = pd.read_csv(path,na_values='?')    
dfAnime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


#### **Tipos de dato adecuados**

In [102]:
dfAnime['name'] = dfAnime['name'].astype("string")
dfAnime['genre'] = dfAnime['genre'].astype("string")
dfAnime['type'] = dfAnime['type'].astype("string")
dfAnime['episodes']=pd.to_numeric(dfAnime.episodes, errors='coerce').dropna().astype(int)
dfAnime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  string 
 2   genre     12232 non-null  string 
 3   type      12269 non-null  string 
 4   episodes  11954 non-null  float64
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(2), int64(2), string(3)
memory usage: 672.5 KB


#### **Busqueda y eliminacion de valores nulos o duplicados**

In [103]:
nullValues = []

for i in range(len(df)):
    if(dfAnime.iloc[i].isnull().sum() != 0):
        print("Null value finded at index: ", i, "\n" , df.iloc[i] , "\n")
        nullValues.append(i)
print("Se encontraron ",len(nullValues)," valores nulos")
#La cantidad de datos con valores nulos no es tan grande en comparacion asi que podemos borrarlos
#Unknown es contado como nulo porque no podemos asumir la cantidad de episodios de un anime en base a otros

Null value finded at index:  74 
 anime_id                                                   21
name                                                One Piece
genre       Action, Adventure, Comedy, Drama, Fantasy, Sho...
type                                                       TV
episodes                                              Unknown
rating                                                   8.58
members                                                504862
Name: 74, dtype: object 

Null value finded at index:  252 
 anime_id                                            235
name                                    Detective Conan
genre       Adventure, Comedy, Mystery, Police, Shounen
type                                                 TV
episodes                                        Unknown
rating                                             8.25
members                                          114702
Name: 252, dtype: object 

Null value finded at index:  615 
 anime_id         

Null value finded at index:  4735 
 anime_id                  32829
name        Nobunaga no Shinobi
genre        Comedy, Historical
type                         TV
episodes                Unknown
rating                     6.65
members                    7207
Name: 4735, dtype: object 

Null value finded at index:  4870 
 anime_id               28921
name        Disney Tsum Tsum
genre                   Kids
type                      TV
episodes             Unknown
rating                  6.61
members                  359
Name: 4870, dtype: object 

Null value finded at index:  4885 
 anime_id                     1960
name           Sore Ike! Anpanman
genre       Comedy, Fantasy, Kids
type                           TV
episodes                  Unknown
rating                       6.61
members                       940
Name: 4885, dtype: object 

Null value finded at index:  4975 
 anime_id                           33240
name        Chi&#039;s Sweet Home (2016)
genre                    

Name: 8136, dtype: object 

Null value finded at index:  8142 
 anime_id          30617
name        Animegatari
genre            Comedy
type              Movie
episodes        Unknown
rating             5.36
members            1308
Name: 8142, dtype: object 

Null value finded at index:  8151 
 anime_id                                 33704
name                               AOTU Shijie
genre       Action, Adventure, Fantasy, Sci-Fi
type                                       ONA
episodes                               Unknown
rating                                     6.0
members                                     79
Name: 8151, dtype: object 

Null value finded at index:  8167 
 anime_id                  33044
name        Asa da yo! Kaishain
genre                    Comedy
type                         TV
episodes                Unknown
rating                     4.33
members                     177
Name: 8167, dtype: object 

Null value finded at index:  8200 
 anime_id               

Null value finded at index:  8938 
 anime_id            31509
name        Ichi-gan Kuni
genre                 NaN
type                Movie
episodes                1
rating               7.17
members                36
Name: 8938, dtype: object 

Null value finded at index:  8955 
 anime_id        28487
name        Ikite Iru
genre             NaN
type              OVA
episodes            1
rating           5.75
members            42
Name: 8955, dtype: object 

Null value finded at index:  8968 
 anime_id                     34502
name               Inazma Delivery
genre       Action, Comedy, Sci-Fi
type                            TV
episodes                        10
rating                         NaN
members                         32
Name: 8968, dtype: object 

Null value finded at index:  8977 
 anime_id             24501
name             Inui-san!
genre       Comedy, Shoujo
type                   OVA
episodes           Unknown
rating                 5.2
members                106
Na

Null value finded at index:  9675 
 anime_id                  33271
name        Nanocore 2nd Season
genre            Action, Sci-Fi
type                        ONA
episodes                Unknown
rating                     5.33
members                     163
Name: 9675, dtype: object 

Null value finded at index:  9689 
 anime_id                          15211
name        Nazotoki-hime wa Meitantei♥
genre                   Mystery, Shoujo
type                                OVA
episodes                        Unknown
rating                             5.62
members                             194
Name: 9689, dtype: object 

Null value finded at index:  9695 
 anime_id            33398
name        Neko Konogoro
genre              Comedy
type                   TV
episodes          Unknown
rating                2.8
members               186
Name: 9695, dtype: object 

Null value finded at index:  9697 
 anime_id                 32572
name        Neko Neko Nihonshi
genre       Historical, 

Null value finded at index:  10474 
 anime_id                                             32663
name        Tama &amp; Friends: Uchi no Tama Shirimasenka?
genre                                      Adventure, Kids
type                                                    TV
episodes                                           Unknown
rating                                                6.67
members                                                 73
Name: 10474, dtype: object 

Null value finded at index:  10498 
 anime_id          32585
name        Telemonster
genre         Adventure
type                 TV
episodes        Unknown
rating              8.5
members              32
Name: 10498, dtype: object 

Null value finded at index:  10575 
 anime_id        30408
name        Tokyo SOS
genre             NaN
type            Movie
episodes            1
rating           2.72
members            87
Name: 10575, dtype: object 

Null value finded at index:  10578 
 anime_id              13457
na

Null value finded at index:  11029 
 anime_id                                                31812
name                 Kuroshitsuji Movie: Book of the Atlantic
genre       Action, Comedy, Demons, Fantasy, Historical, S...
type                                                    Movie
episodes                                                    1
rating                                                    NaN
members                                                 20335
Name: 11029, dtype: object 

Null value finded at index:  11030 
 anime_id                                             34106
name                      Kyoukai no Rinne (TV) 3rd Season
genre       Comedy, Romance, School, Shounen, Supernatural
type                                                    TV
episodes                                           Unknown
rating                                                 NaN
members                                               3628
Name: 11030, dtype: object 

Null value finded at 

Null value finded at index:  11462 
 anime_id                        32864
name        JK to Ero Konbini Tenchou
genre                          Hentai
type                              OVA
episodes                      Unknown
rating                           6.57
members                          2194
Name: 11462, dtype: object 

Null value finded at index:  11496 
 anime_id                 33993
name        300 En no Otsukiai
genre                   Hentai
type                       OVA
episodes               Unknown
rating                    6.53
members                    902
Name: 11496, dtype: object 

Null value finded at index:  11498 
 anime_id                 33153
name        Houkago Initiation
genre                   Hentai
type                       OVA
episodes               Unknown
rating                    6.52
members                   1613
Name: 11498, dtype: object 

Null value finded at index:  11544 
 anime_id                                          32016
name     

In [100]:
dfAnime=dfAnime.drop(nullValues)
dfAnime['episodes'] = dfAnime['episodes'].astype(int)
dfAnime.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11830 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  11830 non-null  int64  
 1   name      11830 non-null  string 
 2   genre     11830 non-null  string 
 3   type      11830 non-null  string 
 4   episodes  11830 non-null  int32  
 5   rating    11830 non-null  float64
 6   members   11830 non-null  int64  
dtypes: float64(1), int32(1), int64(2), string(3)
memory usage: 693.2 KB


In [73]:
duplicados = dfAnime[dfAnime.duplicated()].shape[0]
print("Numero de datos duplicados ",duplicados)

Numero de datos duplicados  0


# **User Rating**

In [9]:
path2 = Path(os.getcwd())
path2 = str(path2.parent.absolute())
path2 = path2+"/datos/rating.csv"
df2 = pd.read_csv(path2,na_values='?')    
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7813737 entries, 0 to 7813736
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memory usage: 178.8 MB
